In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests as rq
import json
import math
import seaborn as sns
from scipy.interpolate import make_interp_spline

In [ ]:
user_id = 557305697713373184

In [ ]:
response = rq.get('https://api.sleeper.app/v1/user/{}/leagues/nfl/2025'.format(user_id))

leagues_from_user = json.loads(response.text)

for league in leagues_from_user:
  print(league['league_id'], league['name'])

In [ ]:
league_id = 1238157874098618368

In [ ]:
# team names
response = rq.get('https://api.sleeper.app/v1/league/{}/users'.format(league_id))

data = json.loads(response.text)

df_users = pd.DataFrame(data)

df_users = pd.concat([df_users, df_users["metadata"].apply(pd.Series)], axis=1)
df_users = df_users.drop(columns=df_users.columns.difference(['user_id','display_name', 'team_name']), axis=1)

df_users['team_name'] = df_users['team_name'].fillna(df_users['display_name'])

# **Data (from API)**

In [ ]:
# rosters

response = rq.get('https://api.sleeper.app/v1/league/{}/rosters'.format(league_id))

data = json.loads(response.text)

df_rosters = pd.DataFrame(data)

df_rosters = pd.concat([df_rosters, df_rosters["settings"].apply(pd.Series)], axis=1)
df_rosters = df_rosters.drop(columns=df_rosters.columns.difference(['wins','roster_id', 'owner_id']), axis=1)

In [ ]:
# Putting them together

df_users = df_users.rename(columns={"user_id": "owner_id"})
result = pd.merge(df_rosters, df_users, on="owner_id")

In [ ]:
# Weekly scores

response = rq.get('https://api.sleeper.app/v1/state/nfl')

data = json.loads(response.text)

current_week = data['week']

for week in range(1, current_week):
  response = rq.get('https://api.sleeper.app/v1/league/{}/matchups/{}'.format(league_id, week))
  data = json.loads(response.text)
  df_week = pd.DataFrame(data)
  df_week = df_week.drop(columns=df_week.columns.difference(['roster_id', 'points']), axis=1)
  df_week = df_week.rename(columns={"points": "week_{}".format(week)})
  result = pd.merge(result, df_week, on="roster_id")

In [ ]:
result.head()

In [ ]:
league = result.to_dict(orient='index')

for team in league:
  league[team]['scores'] = [0 for i in range(current_week-1)]
  league[team]["rank"] = [0 for i in range(current_week-1)]
  league[team]["prob_by_n_wins"] = [0 for i in range(current_week)]
  for week in range(0, current_week-1):
    league[team]['scores'][week] = league[team]['week_{}'.format(week+1)]

for team in league:
  for week in range(0, current_week-1):
    del league[team]['week_{}'.format(week+1)]

num_of_weeks = current_week-1

# **Data Process**

In [ ]:
# Set ranking for each week
for week in range(0, num_of_weeks):
  ordered = dict(sorted(league.items(), key = lambda team: team[1]["scores"][week], reverse=True))
  for index, team in enumerate(ordered.keys()):
    league[team]["rank"][week] = index+1

In [ ]:
from itertools import combinations

sample_list = range(0, num_of_weeks)
w_combinations = list()
for n in range(len(sample_list) + 1):
    w_combinations += list(combinations(sample_list, n))

for wins_array in w_combinations:
  for team in league:
    prob = 1
    for week in range(0, num_of_weeks):
      prob_partial = 1
      if week in wins_array :
        prob_partial *= (len(league)-league[team]["rank"][week])/(len(league)-1.0)
      else:
        prob_partial *= (league[team]["rank"][week]-1)/(len(league)-1.0)
      prob *= prob_partial
    league[team]["prob_by_n_wins"][len(wins_array)] += prob

In [ ]:
def expected_value(values, weights):
    values = np.asarray(values)
    weights = np.asarray(weights)
    return (values * weights).sum()

for team in league:
  league[team]["expected_wins"] = expected_value(league[team]["prob_by_n_wins"], range(0, num_of_weeks+1))
  league[team]["avg"] = np.mean(np.asarray(league[team]["scores"]))
  league[team]["devq"] = np.std(np.asarray(league[team]["scores"]))

In [ ]:
pd.DataFrame(league)

# **Data Exibition**

In [ ]:
names = [0 for i in range (0, len(league))]
averages = [0 for i in range (0, len(league))]
deviations = [0 for i in range (0, len(league))]
wins = [0 for i in range (0, len(league))]
exp_wins = [0 for i in range (0, len(league))]
delta = [0 for i in range (0, len(league))]
weekly_rank = [0 for i in range (0, len(league))]

for index, team in enumerate(league):
  names[index] = league[team]["display_name"]
  averages[index] = round(league[team]["avg"], 1)
  deviations[index] = round(league[team]["devq"], 1)
  wins[index] = league[team]["wins"]
  exp_wins[index] = round(league[team]["expected_wins"], 2)
  delta[index] = round(league[team]["wins"] - league[team]["expected_wins"], 2)
  weekly_rank[index] = league[team]["rank"]

data = {'team': names, 'avg': averages, 'std': deviations, 'wins': wins, 'exp_w': exp_wins, 'delta_w': delta, "weekly_rank": weekly_rank}

df = pd.DataFrame(data)

df = df.sort_values(by=['exp_w'], ascending=False)
df = df.reset_index(drop=True)

df

In [ ]:
df_league = pd.DataFrame(league)
data = df_league.loc["scores"].to_numpy()
names = df_league.loc["team_name"].to_numpy()

fig = plt.figure(figsize =(10, 7))

# Creating axes instance
ax = fig.add_axes([0, 0, 1, 1])

# Creating plot
bp = ax.boxplot(data)

# show plot
plt.title("Boxplot de Pontuação")
plt.xticks(range(1, len(names)+1), names, rotation=90)
plt.show()

In [ ]:
df = df.sort_values(by=['avg'])
df = df.reset_index(drop=True)

plt.errorbar( df['team'], df['avg'], yerr=df['std'], fmt='o', color='Black', elinewidth=3,capthick=3,errorevery=1, alpha=1, ms=4, capsize = 5)
plt.bar(df['team'], df['avg'],tick_label = df['team'])##Bar plot
plt.ylim((70, 165))
plt.xlabel('TEAM') ## Label on X axis
plt.xticks(rotation=90)
plt.ylabel('Average Performance') ##Label on Y axis

In [ ]:
df = df.sort_values(by=['wins', 'exp_w'])
df = df.reset_index(drop=True)

fig, ax = plt.subplots()

ax.bar(df['team'], df['wins'], 0.8, label='Wins', color = 'b')
ax.bar(df['team'], df['exp_w'], 0.3, label='Expected Wins', color = 'c')

ax.set_ylabel('n° of wins')
ax.set_title('Actual Wins & Expected Wins')
ax.legend()

plt.xticks(rotation=90)

plt.show()

In [ ]:
df_league.transpose()

In [ ]:
# Define the team names in an array
team_names = ["Breda SuperBowlers", "Chattanooga CottonPickers", "Utah Pombos", "Stalingrad Farmers"]

# Transpose the DataFrame and create original x values based on the length of the data array for the first team
df = df_league.transpose()
x_original = np.arange(len(df[df["team_name"] == team_names[0]]["prob_by_n_wins"].values[0]))

# Create a smooth x range for interpolation
x_smooth = np.linspace(x_original.min(), x_original.max(), 200)  # 200 points for smoothness

# Plot setup
plt.figure(figsize=(10, 6))  # Optional: Adjust figure size

# Iterate over each team, interpolate, plot, and add a circle marker at the "wins" point
for team in team_names:
    # Get the probability data and number of wins for the team
    data = df[df["team_name"] == team]["prob_by_n_wins"].values[0]
    wins = df[df["team_name"] == team]["wins"].values[0]
    print(team)
    print(data)

    # Create a cubic spline interpolation for the team data
    spline = make_interp_spline(x_original, data, k=2)
    y_smooth = spline(x_smooth)

    # Plot the smoothed curve for the team
    line, = plt.plot(x_smooth, y_smooth, label=team)  # Store the line to get its color

    # Calculate y value for the "wins" x position using the spline
    y_wins = spline(wins)

    # Add a circle marker at the "wins" position on the curve
    plt.plot(wins, y_wins, 'o', color=line.get_color(), markersize=8)

# Final plot adjustments
plt.title("Probabilidade para N vitórias")
plt.xlabel("Número de Vitórias")
plt.ylabel("Probabilidade")
plt.legend()
plt.show()

In [ ]:
# Define the team names in Gamblers
team_names = ["Trentino Quasars", "Athens Spartans", "Varzelandia Gamblers", "Chestnut Flyers"]

# Transpose the DataFrame and create original x values based on the length of the data array for the first team
df = df_league.transpose()
x_original = np.arange(len(df[df["team_name"] == team_names[0]]["prob_by_n_wins"].values[0]))

# Create a smooth x range for interpolation
x_smooth = np.linspace(x_original.min(), x_original.max(), 200)  # 200 points for smoothness

# Plot setup
plt.figure(figsize=(10, 6))  # Optional: Adjust figure size
# Iterate over each team, interpolate, plot, and add a circle marker at the "wins" point
for team in team_names:
    # Get the probability data and number of wins for the team
    data = df[df["team_name"] == team]["prob_by_n_wins"].values[0]
    wins = df[df["team_name"] == team]["wins"].values[0]

    # Create a cubic spline interpolation for the team data
    spline = make_interp_spline(x_original, data, k=2)
    y_smooth = spline(x_smooth)

    # Plot the smoothed curve for the team
    line, = plt.plot(x_smooth, y_smooth, label=team)  # Store the line to get its color

    # Calculate y value for the "wins" x position using the spline
    y_wins = spline(wins)

    # Add a circle marker at the "wins" position on the curve
    plt.plot(wins, y_wins, 'o', color=line.get_color(), markersize=8)

# Final plot adjustments
plt.title("Probabilidade para N vitórias")
plt.xlabel("Número de Vitórias")
plt.ylabel("Probabilidade")
plt.legend()
plt.show()

In [ ]:
# Define the team names in an array
team_names = ["Vespasiano JetEagles", "Sabinópolis Roludos", "Dusseldorf Vetter's", "Indianapolis Foxes"]

# Transpose the DataFrame and create original x values based on the length of the data array for the first team
df = df_league.transpose()
x_original = np.arange(len(df[df["team_name"] == team_names[0]]["prob_by_n_wins"].values[0]))

# Create a smooth x range for interpolation
x_smooth = np.linspace(x_original.min(), x_original.max(), 200)  # 200 points for smoothness

# Plot setup
plt.figure(figsize=(10, 6))  # Optional: Adjust figure size

# Iterate over each team, interpolate, plot, and add a circle marker at the "wins" point
for team in team_names:
    # Get the probability data and number of wins for the team
    data = df[df["team_name"] == team]["prob_by_n_wins"].values[0]
    wins = df[df["team_name"] == team]["wins"].values[0]

    # Create a cubic spline interpolation for the team data
    spline = make_interp_spline(x_original, data, k=2)
    y_smooth = spline(x_smooth)

    # Plot the smoothed curve for the team
    line, = plt.plot(x_smooth, y_smooth, label=team)  # Store the line to get its color

    # Calculate y value for the "wins" x position using the spline
    y_wins = spline(wins)

    # Add a circle marker at the "wins" position on the curve
    plt.plot(wins, y_wins, 'o', color=line.get_color(), markersize=8)

# Final plot adjustments
plt.title("Probabilidade para N vitórias")
plt.xlabel("Número de Vitórias")
plt.ylabel("Probabilidade")
plt.legend()
plt.show()

In [ ]:
qb_data = {'QB': ["Indiana Jones", "JJ McCarthy"], 'PassingYards': [1078, 301]}
df_qb = pd.DataFrame(data=qb_data)

fig, ax = plt.subplots()

ax.bar(df_qb['QB'], df_qb['PassingYards'], 0.8, label='Passing Yards', color = 'b')

ax.set_ylabel("Quarterback")
ax.set_title('Aposta Lucca vs Cesinha')
ax.legend()


plt.show()